# ***Setup***

In [1]:
!pip install python-dotenv groq bitsandbytes chromadb pypdf -U langchain-community langchain_groq langchain_cohere sentence-transformers
from IPython.display import clear_output
clear_output()

# ***Import Libraries***

In [2]:
%%writefile -a utils.py
from dotenv import load_dotenv
import os
from huggingface_hub import login
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain_groq import ChatGroq
from langchain import hub
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline
import torch

Writing utils.py


In [3]:
%%writefile -a utils.py
load_dotenv("/kaggle/input/images/API.env")

Appending to utils.py


In [4]:
%%writefile -a utils.py
groq_api_key = os.getenv("GROQ_API_KEY")
huggingface_token = os.getenv("HUGGING_FACE_TOKEN")
cohere_api_key = os.getenv("COHERE_API_KEY")
ngrok_token = os.getenv("NGROK_TOKEN")

Appending to utils.py


# ***Data Ingestions***

In [5]:
%%writefile -a utils.py
loader = PyPDFLoader('/kaggle/input/images/Introduction to Cybersecurity.pdf')
documents = loader.load()

Appending to utils.py


In [6]:
%%writefile -a utils.py
print("The number of pages is",len(documents))

Appending to utils.py


In [7]:
%%writefile -a utils.py
for doc in documents[13:20]:
    print(f"page_content_for_page_{doc.metadata['page_label']} is\n {doc.page_content[:1000]}...")
    print("="*80)
    

Appending to utils.py


# ***Chunking***

In [8]:
%%writefile -a utils.py
# Define the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Number of characters in each chunk
    chunk_overlap=200,    # Overlap between consecutive chunks
)

# Apply splitting to the documents
chunks = text_splitter.split_documents(documents)
print(f"\nNumber of chunks after splitting: {len(chunks)}")
print("="*80)
# Display the first chunk
print(f"First chunk: {chunks[0].page_content}...")
print("="*80)
print(f"Number of characters in the first chunk: {len(chunks[0].page_content)}")

Appending to utils.py


# ***Create Embeddings***

In [9]:
%%writefile -a utils.py
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Appending to utils.py


# ***Create Database***

In [10]:
%%writefile -a utils.py
persist_directory = '/kaggle/working/chroma_db/'

langchain_chroma = Chroma.from_documents(documents=chunks,embedding=embedding_model,persist_directory=persist_directory,
                                         collection_name='Cybersecurity')
print("Collection Created")

Appending to utils.py


# ***Load Model***

In [11]:
%%writefile -a utils.py
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model="llama-3.1-8b-instant",
    temperature=0.0
)

Appending to utils.py


In [24]:
login(huggingface_token)  

In [25]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [26]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [27]:
query_pipline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.5,
    return_full_text=False,
    repetition_penalty=1.1,
    top_p= 0.9
)
llm = HuggingFacePipeline(pipeline=query_pipline)

Device set to use cuda:0
/tmp/ipykernel_35/1114573771.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipline)


In [62]:
prompt = ChatPromptTemplate.from_template("""
You are a helpful and concise AI assistant specialized in cybersecurity topics based on the Cisco Networking Academy's "Introduction to Cybersecurity" material.

If the user's message is a **greeting** (like "Hi", "Hello", "Hey"), respond with a **friendly welcome message** and do not use the context.

ONLY use the context below to answer if the user asks a factual question. If the answer is not in the context, respond with:
"The information is not available in the provided context."

DO NOT explain your reasoning.
DO NOT repeat yourself.
DO NOT justify.
Just answer in one short paragraph.

Context:
{context}

Question:
{question}

Answer:
""")

# prompt = ChatPromptTemplate.from_template("""
# You are a helpful and informative AI assistant specialized in cybersecurity topics based on the Cisco Networking Academy's "Introduction to Cybersecurity" material.

# If the user's message is a **greeting** (like "Hi", "Hello", "Hey"), respond with a **friendly welcome message** and do not use the context.

# Use ONLY the context below to answer if the user asks a factual question. If the answer is not in the context, respond with:
# "The information is not available in the provided context."

# DO NOT guess.
# DO NOT add information beyond the context.
# DO NOT repeat yourself.

# Provide a **detailed, clear, and informative answer** based strictly on the context.

# Context:
# {context}

# Question:
# {question}

# Answer:
# """)

# ***Retrieve Rerank***

In [12]:
%%writefile -a utils.py
# Step 2: Create retriever (assuming you have langchain_chroma ready)
retriever = langchain_chroma.as_retriever(search_kwargs={"k": 3})

compressor = CohereRerank(model="rerank-english-v3.0",cohere_api_key=cohere_api_key)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

Appending to utils.py


In [13]:
%%writefile -a utils.py
docs = compression_retriever.get_relevant_documents("Where is personal data usually stored?")
for doc in docs:
    print(doc.page_content)
    print("-" * 50)  


Appending to utils.py


# ***RAG without Chat History***

In [63]:
# # Step 3: Build the RAG QA chain using the custom prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    return_source_documents=True,  # optional if you want to see sources
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},  # use the strong prompt here
)

In [69]:
# Step 4: Query the RAG model
# query = "What service did Google introduce to search TV programs?"
# query = "What did Technorati estimate about the number of blogs?"
# query = "What is 'Explainable AI' and why is it considered important?"
query = "What is two-factor authentication and how does it work?"
# query = "hello! Can you help me with something?"
# query = "What causes cancer and how to avoid it?"
result = qa_chain.invoke({"query": query})
# result = qa_chain({"query": query})

# Step 5: Print the response and sources for traceability
response = result["result"]
sources = result["source_documents"]

print(f"Query: {query}")
print(f"Response: {response}\n")

print("Sources:")
for idx, doc in enumerate(sources, 1):
    print(f"{idx}. {doc.metadata.get('tags', 'No Tags')} - {doc.page_content[:200]}...")

Query: What is two-factor authentication and how does it work?
Response: Two-factor authentication (2FA) is a security process that requires two different forms of authentication to verify a user's identity. This method significantly enhances security by making it more difficult for unauthorized users to access sensitive information.

Here’s how it works:

1. **Username/Password or PIN**: The first factor typically involves something the user knows, such as a username and password or a personal identification number (PIN).

2. **Second Token**: The second factor involves something the user has or is, which can be:
   - A physical object, like a credit card, ATM card, phone, or fob.
   - A biometric scan, such as a fingerprint, palm print, or facial and voice recognition.

By combining these two factors, 2FA ensures that even if one factor is compromised (for example, if a password is stolen), the attacker still needs the second factor to gain access. This dual-layered approach signific

# ***RAG with Chat History***

In [14]:
%%writefile -a utils.py
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, compression_retriever, contextualize_q_prompt
)

Appending to utils.py


In [15]:
%%writefile -a utils.py
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

Appending to utils.py


In [16]:
%%writefile -a utils.py
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

Appending to utils.py


In [17]:
%%writefile -a utils.py
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

Appending to utils.py


In [18]:
%%writefile -a utils.py
def ChatResponse(question,session_id):
    
    response = conversational_rag_chain.invoke(
    {"input": question},
    config={
        "configurable": {"session_id": session_id}
    },
)["answer"]
    
    return response
    

Appending to utils.py


In [23]:
response = ChatResponse("What is cybersecurity?","abc123")
print(response)

Cybersecurity refers to the protection of networked systems and data from unauthorized use or harm.


In [24]:
response = ChatResponse("Why is it important?","abc123")
print(response)

Cybersecurity is important because it helps protect your data and privacy from being compromised by cyber threats, such as hacking and sabotage, which can have severe consequences.


In [25]:
response = ChatResponse("What last 2 questions did I ask?","abc123")
print(response)

Your last two questions were: 
1. What is cybersecurity?
2. Why is it important?


In [26]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is the difference between online and offline identity?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your online identity refers to the persona you present while interacting in cyberspace, whereas your offline identity is the one that interacts in person at home, school, or work. Your online identity should only reveal a limited amount of information about you, using a username or alias that is appropriate and respectful. In contrast, your offline identity is your real-life persona, with no need to hide or limit personal information.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What kind of risks are associated with your online identity?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your online identity is at risk of revealing too much personal information, which can attract unwanted attention. Using a username or alias that includes personal inform

# ***Deployment***

In [20]:
from IPython.display import clear_output

In [19]:
!pip install --upgrade streamlit
clear_output()

In [20]:
%%writefile app.py
import streamlit as st
from utils import ChatResponse
    
# Page config
st.set_page_config(page_title="🛡️ Cyber RAG Chatbot", layout="wide")
st.markdown("""
    <style>
        body { background-color: #0d1117; color: white; }
        .user-msg {
            background-color: #1f6feb;
            padding: 10px;
            border-radius: 10px;
            margin-bottom: 10px;
            color: white;
        }
        .assistant-msg {
            background-color: #e6f4ea;  /* Light pastel green */
            padding: 10px;
            border-radius: 10px;
            margin-bottom: 10px;
            border-left: 4px solid #34d058;  /* Bright green border */
            color: #1b1f23;  /* Dark text for contrast */
        }
        .chatbox {
            max-height: 500px;
            overflow-y: auto;
            padding-right: 10px;
        }
        .stTextInput > div > div > input {
            background-color: #161b22;
            color: white;
        }
    </style>
""", unsafe_allow_html=True)

# Title
st.title("🛡️ Cybersecurity RAG Chatbot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Welcome to the Cybersecurity Assistant. How can I help you today?"}
    ]

# Sidebar
with st.sidebar:
    st.header("📄 Upload Files")
    uploaded_files = st.file_uploader("Upload PDF or TXT files", type=["pdf", "txt"], accept_multiple_files=True)

    st.markdown("---")
    st.subheader("⚙️ Settings")
    st.slider("Top-k Chunks", 1, 10, 3)
    st.slider("Temperature", 0.0, 1.0, 0.3)

# Chat display
st.markdown("### 💬 Chat")
st.markdown('<div class="chatbox">', unsafe_allow_html=True)
for msg in st.session_state.messages:
    if msg["role"] == "user":
        st.markdown(f'<div class="user-msg">🧑‍💻 <b>You:</b><br>{msg["content"]}</div>', unsafe_allow_html=True)
    else:
        st.markdown(f'<div class="assistant-msg">🤖 <b>Assistant:</b><br>{msg["content"]}</div>', unsafe_allow_html=True)
st.markdown('</div>', unsafe_allow_html=True)

# Input box
user_input = st.text_input("Type your question securely:")

if st.button("📤 Send") and user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})
    assistant_response = ChatResponse(user_input,"abc123")
    st.session_state.messages.append({"role": "assistant", "content": assistant_response})
    st.rerun()


Writing app.py


In [21]:
!wget -qO - https://ipv4.icanhazip.com

35.189.38.213


In [22]:
!npm install -g localtunnel
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
added 22 packages in 2s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹npm notice
npm notice New major version of npm available! 10.8.2 -> 11.4.2
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.4.2
npm notice To update run: npm install -g npm@11.4.2
npm notice
⠹⠙⠹⠸⠼⠴


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://35.189.38.213:8501

your url is: https://fast-squids-serve.loca.lt
/kaggle/working/utils.py:19: LangChainDeprecationWarning: Importing HuggingFaceEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import HuggingFaceEmbeddings

with new imports of:

>> from langchain_community.embeddings import HuggingFaceEmbeddings
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/vers